In [36]:
import os
import re

In [37]:
#Celula de código para tratar os testes de conformidade e substituir Labels por offset

def transformar_labels_e_exit_em_linhas(asm_codigo: str) -> str:
    linhas = asm_codigo.splitlines()
    mapeamento_labels = {}
    resultado_asm = []
    cont = 0

    # Determinar a localização dos rótulos e do "exit"
    #for indice, linha in enumerate(linhas, start=1):
    for indice, linha in enumerate(linhas):
        # Remover comentários na linha
        linha = linha.split("#", 1)[0].strip()
        if not linha or linha.startswith("--"):  # Ignorar linhas vazias ou comentários
            continue
        if linha.endswith(":"):
            label = linha[:-1]  # Captura o nome da label sem os dois pontos
            mapeamento_labels[label] = cont  # Mapear o rótulo para o número da linha
        elif "exit" in linha:
            mapeamento_labels["exit"] = cont  # Adiciona o mapeamento para exit
        cont +=1

    cont = 0
    # Substituir os rótulos e o "exit" por números de linha
    for linha in linhas:
        # Remover comentários na linha
        linha = linha.split("#", 1)[0].strip()
        if not linha or linha.startswith("--"):  # Ignorar linhas vazias ou comentários
            continue
        cont += 1
        linha_formatada = linha
        for label, numero_linha in mapeamento_labels.items():
            if f"{label}" in linha:
                if label == "exit":
                    continue
                #linha_formatada = linha_formatada.replace(f"{label}", f"{numero_linha-cont-1}")
                linha_formatada = re.sub(rf'\b{label}\b', f'{numero_linha-cont}',linha_formatada)

        if not linha_formatada.endswith(":"):  # Ignorar linhas que são apenas rótulos
            resultado_asm.append(linha_formatada)

    return "\n".join(resultado_asm)

# Exemplo de uso
entrada_asm = """mov %r2, 0xe
ldxh %r3, [%r1+12]
jne %r3, 0x81, L1
mov %r2, 0x12
ldxh %r3, [%r1+16]
and %r3, 0xffff
L1:
jne %r3, 0x8, L2
add %r1, %r2
mov %r0, 0x1
ldxw %r1, [%r1+16]
and %r1, 0xffffff
jeq %r1, 0x1a8c0, exit
L2:
mov %r0, 0x0
exit"""

entrada_asm1 ="""# Copyright (c) Big Switch Networks, Inc
# SPDX-License-Identifier: Apache-2.0
-- asm
mov %r1, 0
ja L2
L1:
mov %r2, 0
exit
L2:
mov %r0, 0
ja L1
-- result
0x0"""


saida_asm = transformar_labels_e_exit_em_linhas(entrada_asm1)
print(saida_asm)

mov %r1, 0
ja 3
mov %r2, 0
exit
mov %r0, 0
ja -6
0x0


In [38]:

#Celula de Código que define a implementação do parser que cria um arquivo .lean funcional para cada teste

def parseTestsImport(conteudo_data: str) -> str:
    # Separar as seções da string de entrada
    cabecalho = """import «ProjetoConformance».ConformanceTests
"""
    cabecalhoOld = """import «ProjetoConformance».ConformanceTests
    structure MemorySpace where
    data : Fin 512 → Nat \n
def emptyMemory : MemorySpace := {data := fun _ => 0}
"""
    
    partes = conteudo_data.split("--")
    mem_secao = None
    asm_secao = None
    result_secao = None

    for parte in partes:
        if parte.strip().startswith("mem"):
            mem_secao = parte.split("\n", 1)[1].strip()  # Obter apenas o conteúdo após "mem"
        elif parte.strip().startswith("asm"):
            asm_secao = parte.split("\n", 1)[1].strip()  # Obter a parte ASM
        elif parte.strip().startswith("result"):
            result_secao = parte.split("\n", 1)[1].strip()  # Obter o conteúdo do resultado
    
    if not mem_secao :
        mem_secao = "00 00 00 00 00 00 00 00"

    # Criar o formato .lean
    #if not mem_secao or not asm_secao or not result_secao:
        #return "Erro: Seções necessárias não foram encontradas no arquivo de entrada."
    

    # Formatar a memória para o formato lean
    mem_formatada = "{mem|\n" + "\n".join(
        [" ".join(f"'{char}'" for char in linha.strip() if char.strip()) for linha in mem_secao.splitlines()]
    ) + " }"

    asm_secao = transformar_labels_e_exit_em_linhas(asm_secao)

    #eval exeConformance {{exe|\n# Copyright Big Switch Networks Inc
    # Formatar o asm e o resultado para .lean
    lean_output = f"""{cabecalho.strip()}
    \ndef memory :=
    createStackMemory 0 emptyMemory  \n{mem_formatada}
    \n#eval exeConformanceCompareResult {{exe|\n# Copyright Big Switch Networks Inc

# SPDX_License_Identifier Apache_2_0  \nasm  \n{asm_secao}  \nresult   \n{result_secao}   \n}} memory
    """
    return lean_output.strip()


def ler_arquivos_data_e_transformarImport(diretorio):
    # Verificar se o diretório existe
    if not os.path.exists(diretorio):
        print("O diretório especificado não existe.")
        return
    
    # Criar diretório para os arquivos .lean, se não existir
    destino = os.path.join(diretorio, "..\lean_data_Import")
    os.makedirs(destino, exist_ok=True)

    # Percorrer arquivos no diretório
    for arquivo in os.listdir(diretorio):
        if arquivo.endswith(".data"):  # Verifica extensão .data
            caminho_arquivo = os.path.join(diretorio, arquivo)
            
            # Ler conteúdo como string
            with open(caminho_arquivo, "r", encoding="utf-8") as f:
                conteudo = f.read()
                conteudoLean = parseTestsImport(conteudo)
            
            # Transformar nome do arquivo .data em .lean
            nome_arquivo_lean = os.path.splitext(arquivo)[0] + ".lean"
            caminho_arquivo_lean = os.path.join(destino, nome_arquivo_lean)
            
            # Salvar string como arquivo .lean
            with open(caminho_arquivo_lean, "w", encoding="utf-8") as f:
                f.write(conteudoLean)
            
            print(f"Arquivo {arquivo} transformado em {nome_arquivo_lean} com sucesso!")

<>:60: SyntaxWarning: invalid escape sequence '\l'
<>:60: SyntaxWarning: invalid escape sequence '\l'
C:\Users\marco\AppData\Local\Temp\ipykernel_22100\84054480.py:60: SyntaxWarning: invalid escape sequence '\l'
  destino = os.path.join(diretorio, "..\lean_data_Import")


In [39]:

#Celula de Código que define a implementação do parser que cria um arquivo .lean funcional para todos os testes

def parseTestsImportUnico(conteudo_data: str,nome_original: str) -> str:
    # Separar as seções da string de entrada
    
    partes = conteudo_data.split("--")
    mem_secao = None
    asm_secao = None
    result_secao = None

    for parte in partes:
        if parte.strip().startswith("mem"):
            mem_secao = parte.split("\n", 1)[1].strip()  # Obter apenas o conteúdo após "mem"
        elif parte.strip().startswith("asm"):
            asm_secao = parte.split("\n", 1)[1].strip()  # Obter a parte ASM
        elif parte.strip().startswith("result"):
            result_secao = parte.split("\n", 1)[1].strip()  # Obter o conteúdo do resultado
    
    if not mem_secao :
        mem_secao = "00 00 00 00 00 00 00 00"

    # Criar o formato .lean
    #if not mem_secao or not asm_secao or not result_secao:
        #return "Erro: Seções necessárias não foram encontradas no arquivo de entrada."
    

    # Formatar a memória para o formato lean
    mem_formatada = "{mem|\n" + "\n".join(
        [" ".join(f"'{char}'" for char in linha.strip() if char.strip()) for linha in mem_secao.splitlines()]
    ) + " }"
    
    asm_secao = transformar_labels_e_exit_em_linhas(asm_secao)
    nome_original = nome_original.replace("-by","").replace("-","_")

    #eval exeConformance {{exe|\n# Copyright Big Switch Networks Inc
    # Formatar o asm e o resultado para .lean
    lean_output = f"""
    \n------------------------------
    \ndef memory{nome_original} :=
    createStackMemory 0 emptyMemory  \n{mem_formatada}\n
def prog{nome_original} : TestEval :=
{{exe|\n# Copyright Big Switch Networks Inc\n# SPDX_License_Identifier Apache_2_0
asm  \n{asm_secao}  \nresult   \n{result_secao}   \n}}\n
-- #eval exeConformanceCompareResult prog{nome_original} memory{nome_original}
-- #eval exeConformanceDebug prog{nome_original} memory{nome_original}
\n  
------------------------------
"""
    return lean_output.strip()


def ler_arquivos_data_e_transformarArquivoUnico(diretorio):
    # Verificar se o diretório existe
    if not os.path.exists(diretorio):
        print("O diretório especificado não existe.")
        return
    
    # Criar diretório para os arquivos .lean, se não existir
    destino = os.path.join(diretorio, "..\lean_data_Import_Unico")
    os.makedirs(destino, exist_ok=True)
    
    conteudoFinal = ""
    cabecalho = "import «ProjetoConformance».ConformanceTests\n \n \n"

    conteudoFinal = conteudoFinal + cabecalho

    # Percorrer arquivos no diretório
    for arquivo in os.listdir(diretorio):
        if arquivo.endswith(".data"):  # Verifica extensão .data
            caminho_arquivo = os.path.join(diretorio, arquivo)
            
            # Ler conteúdo como string
            with open(caminho_arquivo, "r", encoding="utf-8") as f:
                conteudo = f.read()
                nome_original = os.path.splitext(arquivo)[0]
                conteudoLean = parseTestsImportUnico(conteudo,nome_original)
                conteudoFinal = conteudoFinal + conteudoLean
            
    # Transformar nome do arquivo .data em .lean
    nome_arquivo_lean = os.path.splitext(arquivo)[0] + ".lean"
    caminho_arquivo_lean = os.path.join(destino, "conformanceTestsUnico.Lean")
            
    # Salvar string como arquivo .lean
    
    with open(caminho_arquivo_lean, "w", encoding="utf-8") as f:
        f.write(conteudoFinal)
            
    print(f"Arquivos transformados em conformanceTestsUnico com sucesso!")

<>:59: SyntaxWarning: invalid escape sequence '\l'
<>:59: SyntaxWarning: invalid escape sequence '\l'
C:\Users\marco\AppData\Local\Temp\ipykernel_22100\2246891059.py:59: SyntaxWarning: invalid escape sequence '\l'
  destino = os.path.join(diretorio, "..\lean_data_Import_Unico")


In [40]:
#Celula de Código que define a implementação do parser que cria um arquivo .lean funcional para cada teste usando def

def parseTests(conteudo_data: str, nome_original: str) -> str:
    # Cabeçalho para o arquivo Lean
    cabecalho = f"import «ProjetoConformance».ConformanceTests\n\n"
    
    # Processar as seções da string de entrada
    partes = conteudo_data.split("--")
    mem_secao = None
    asm_secao = None
    result_secao = None

    for parte in partes:
        if parte.strip().startswith("mem"):
            mem_secao = parte.split("\n", 1)[1].strip()
        elif parte.strip().startswith("asm"):
            asm_secao = parte.split("\n", 1)[1].strip()
        elif parte.strip().startswith("result"):
            result_secao = parte.split("\n", 1)[1].strip()

    if not mem_secao:
        mem_secao = "00 00 00 00 00 00 00 00"

    # Formatando mem_secao
    mem_formatada = "{mem|\n" + "\n".join(
        [" ".join(f"'{char}'" for char in linha.strip() if char.strip()) for linha in mem_secao.splitlines()]
    ) + " }"

    # Formatando asm_secao
    asm_secao = transformar_labels_e_exit_em_linhas(asm_secao)  # Função fictícia assumida como existente.

    # Novo padrão de saída usando o nome original do arquivo
    nome_def = f"prog{nome_original}"
    nome_def = nome_def.replace("-by","").replace("-","_")
    nome_def_mem = f"memory{nome_original}"
    nome_def_mem = nome_def_mem.replace("-by","").replace("-","_")
    lean_output = f"""
\n------------------------------
\ndef {nome_def_mem} :=
createStackMemory 0 emptyMemory  \n{mem_formatada}
def {nome_def} : TestEval := 
{{exe|
# Copyright Big Switch Networks Inc
# SPDX_License_Identifier Apache_2_0
asm
{asm_secao}
result
{result_secao}
}}"""
    return lean_output.strip(), nome_def, nome_def_mem

def ler_arquivos_data_e_transformar(diretorio):
    if not os.path.exists(diretorio):
        print("O diretório especificado não existe.")
        return
    
    destino = os.path.join(diretorio, "..\lean_data")
    os.makedirs(destino, exist_ok=True)


    for arquivo in os.listdir(diretorio):
        if arquivo.endswith(".data"):
            caminho_arquivo = os.path.join(diretorio, arquivo)

            with open(caminho_arquivo, "r", encoding="utf-8") as f:
                conteudo = f.read()
                nome_original = os.path.splitext(arquivo)[0]
                conteudoLean, nome_def, nome_def_mem = parseTests(conteudo, nome_original)

                nome_arquivo_lean = f"{nome_original}.lean"
                caminho_arquivo_lean = os.path.join(destino, nome_arquivo_lean)

                with open(caminho_arquivo_lean, "w", encoding="utf-8") as f_lean:
                    f_lean.write(conteudoLean)

                
                print(f"Arquivo {arquivo} transformado em {nome_arquivo_lean} com sucesso!")

<>:57: SyntaxWarning: invalid escape sequence '\l'
<>:57: SyntaxWarning: invalid escape sequence '\l'
C:\Users\marco\AppData\Local\Temp\ipykernel_22100\1169356574.py:57: SyntaxWarning: invalid escape sequence '\l'
  destino = os.path.join(diretorio, "..\lean_data")


In [41]:
import os

def combinar_arquivos_lean(diretorio, destino_arquivo):
    # Verificar se o diretório existe
    if not os.path.exists(diretorio):
        print("O diretório especificado não existe.")
        return
    
    # Criar pasta de destino, se necessário
    destino_pasta = os.path.dirname(destino_arquivo)
    if destino_pasta and not os.path.exists(destino_pasta):
        os.makedirs(destino_pasta, exist_ok=True)

    # Lista para guardar os nomes dos defs
    nomes_defs_prog = []
    nomes_defs_memory = []

    # Criar o arquivo consolidado
    with open(destino_arquivo, "w", encoding="utf-8") as arquivo_saida:
        # Adicionar o cabeçalho no início do arquivo consolidado
        arquivo_saida.write("import «ProjetoConformance».ConformanceTests\n\n")
        
        # Combinar arquivos .lean
        for arquivo in os.listdir(diretorio):
            if arquivo.endswith(".lean"):
                caminho_arquivo = os.path.join(diretorio, arquivo)
                
                # Ler o conteúdo do arquivo atual
                with open(caminho_arquivo, "r", encoding="utf-8") as arquivo_atual:
                    conteudo = arquivo_atual.read()

                    # Adicionar comentário indicando o início do arquivo
                    arquivo_saida.write(f"-- Início do arquivo: {arquivo}\n")
                    arquivo_saida.write(conteudo)
                    arquivo_saida.write(f"\n-- Fim do arquivo: {arquivo}\n\n")
                
                # Extrair e salvar o nome do def a partir do nome do arquivo
                nome_def = f"{os.path.splitext(arquivo)[0]}"
                nome_def = nome_def.replace("-by","").replace("-","_")
                nomes_defs_prog.append("prog"+nome_def)
                nomes_defs_memory.append("memory"+nome_def)

        # Adicionar a lista de nomes dos programas ao final do arquivo consolidado
        arquivo_saida.write("-- Lista de defs realizados\n")
        arquivo_saida.write("def prog_list := [\n")
        arquivo_saida.write(",\n".join(f'{nome}' for nome in nomes_defs_prog))
        arquivo_saida.write("\n]\n")
        # Adicionar a lista de nomes dos imputs ao final do arquivo consolidado
        arquivo_saida.write("def memory_list := [\n")
        arquivo_saida.write(",\n".join(f'{nome}' for nome in nomes_defs_memory))
        arquivo_saida.write("\n]\n")

    print(f"Todos os arquivos .lean foram combinados em {destino_arquivo}, incluindo os nomes dos defs no final.")

In [42]:
diretorio_entrada="""tests\data"""
diretorio_entrada_Lean="""tests\lean_data"""
diretorio_saida_Lean= os.path.join("ProjetoConformance", "conformance_Lean.lean")
ler_arquivos_data_e_transformar(diretorio_entrada)
ler_arquivos_data_e_transformarImport(diretorio_entrada)
ler_arquivos_data_e_transformarArquivoUnico(diretorio_entrada)
combinar_arquivos_lean(diretorio_entrada_Lean,diretorio_saida_Lean)

Arquivo add.data transformado em add.lean com sucesso!
Arquivo add64.data transformado em add64.lean com sucesso!
Arquivo alu-arith.data transformado em alu-arith.lean com sucesso!
Arquivo alu-bit.data transformado em alu-bit.lean com sucesso!
Arquivo alu64-arith.data transformado em alu64-arith.lean com sucesso!
Arquivo alu64-bit.data transformado em alu64-bit.lean com sucesso!
Arquivo arsh32-imm-high.data transformado em arsh32-imm-high.lean com sucesso!
Arquivo arsh32-imm-neg.data transformado em arsh32-imm-neg.lean com sucesso!
Arquivo arsh32-imm.data transformado em arsh32-imm.lean com sucesso!
Arquivo arsh32-reg-high.data transformado em arsh32-reg-high.lean com sucesso!
Arquivo arsh32-reg-neg.data transformado em arsh32-reg-neg.lean com sucesso!
Arquivo arsh32-reg.data transformado em arsh32-reg.lean com sucesso!
Arquivo arsh64-imm-high.data transformado em arsh64-imm-high.lean com sucesso!
Arquivo arsh64-imm-neg.data transformado em arsh64-imm-neg.lean com sucesso!
Arquivo arsh

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\l'
<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\l'
C:\Users\marco\AppData\Local\Temp\ipykernel_22100\132707386.py:1: SyntaxWarning: invalid escape sequence '\d'
  diretorio_entrada="""tests\data"""
C:\Users\marco\AppData\Local\Temp\ipykernel_22100\132707386.py:2: SyntaxWarning: invalid escape sequence '\l'
  diretorio_entrada_Lean="""tests\lean_data"""


Arquivo stb.data transformado em stb.lean com sucesso!
Arquivo stdw.data transformado em stdw.lean com sucesso!
Arquivo sth.data transformado em sth.lean com sucesso!
Arquivo stw.data transformado em stw.lean com sucesso!
Arquivo stxb-all.data transformado em stxb-all.lean com sucesso!
Arquivo stxb-all2.data transformado em stxb-all2.lean com sucesso!
Arquivo stxb-chain.data transformado em stxb-chain.lean com sucesso!
Arquivo stxb.data transformado em stxb.lean com sucesso!
Arquivo stxdw.data transformado em stxdw.lean com sucesso!
Arquivo stxh.data transformado em stxh.lean com sucesso!
Arquivo stxw.data transformado em stxw.lean com sucesso!
Arquivo subnet.data transformado em subnet.lean com sucesso!
Arquivo swap16.data transformado em swap16.lean com sucesso!
Arquivo swap32.data transformado em swap32.lean com sucesso!
Arquivo swap64.data transformado em swap64.lean com sucesso!
Arquivo add.data transformado em add.lean com sucesso!
Arquivo add64.data transformado em add64.lean co